# 파일은 없지만 코드만(음주 분석)

%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

file_path = './data/drinks.csv'
drinks = pd.read_csv(file_path)

print(drinks.info())

drinks.head()

drinks.describe()

# feature 간 상관관계 탐색: 단순상관분석(선형관계) vs. 다중상관분석

# 단순상관분석 중 가장 보편적인 방법: 피어슨 상관계수

corr = drinks[['beer_servings', 'wine_servings']].corr(method = 'pearson')
print(corr) # 2x2 행렬로 상관계수 나옴

cols = ['beer_servings', 'wine_servings', 'spirit_servings']
corr = drinks[cols].corr(method = 'pearson')
print(corr)

# 행렬 상관관계분석 결과를 시각화

In [1]:
!pip install seaborn

# 히트맵 그리기

import seaborn as sns

cols_view = ['beer', 'spirit','wine','alchol']
sns.set(fron_scale = 1.5)
hm = sns.heatmap(corr.values, # 이전에 예제에는 판다스 혹은 넘파이 타입의 데이터프레임이 들어갔었음. 여기서는 그대신 행렬 넣음, 행렬도 결국엔 데이터프레임 타입
            cbar = True,
            annot = True,
            square = True,
            fmt = '.2f',
            annot_kws = {'size':15},
            yticklabels = cols_view,
            xticklabels = cols_view)
plt.tight_layout()
plt.show()

# scatter plot 그리기

sns.set(style = 'whitegrid', context ='notebook')
sns.pairplot(drinks[['beer_servings', 'spirit_servings', 'wine_servings']], height = 2.5) # 대괄호를 두 번 써줘야 함

plt.show()


# 결측데이터 처리

print(drinks.isnull().sum())
print(drinks.dtypes) # 아마도 continents에 결측데이터 23개가 나오나봄

drinks['continent'] = drinks['continent'].fillna('OT') # na(결측값)을 OT 로 채워줘라라는 의미

drinks['continent'].isnull().sum() # 결측값 개수인가봄 OT로 바꿔주면 결측값 0으로 나옴

drinks['continent'].value_counts() # 테이블 형태로 출력

# 파이차트 그리기

labels = drinks('continent').value_counts().index.tolist()
fracs1 = drinks('continent').value_counts().values.tolist()
explode = (0, 0, 0, 0.25, 0, 0)

plt.pie(fracs1, explode = explode, labels = labels, autopct = '%.0f%%', shadow = True)
plt.title('null data to \'OT\'')
plt.show()


# 그룹 단위의 데이터 분석: 데이터 분석

result = drinks.groupby('continent').spirit_servings.agg(['mean', 'min', 'max', 'sum']) # agg는 그룹별로 각각 정의해준 연산을 리스트로 한번에 요청하는 함수
result = drinks.groupby('continent')['spirit_servings'].agg(['mean', 'min', 'max', 'sum']) # 위와 똑같은 문법임
result.head()

# 좀 더 복잡한 분석. 전체 평균보다 더 많은 알콜을 섭취하는 대륙

total_mean = drinks.total_litres_of_pure_alcohol.mean()
continent_mean = drinks.groupby('continent')['total_litres_of_pure_alcohol'].mean()
continent_over_mean = continent_mean[continent_mean >= total_mean]
print(continent_over_mean)

# 평균 beer_servings 가장 높은 대륙

beer_continent = drinks.groupby('continent').beer_servings.mean().idxmax() # idxmax 가장 값이 큰 인덱스를 구하겠다는 함수. idxmax 없으면 그냥 대륙의 평균 beer_servings
print(beer_continent)

# 막대그래프 그리기

n_groups = len(result.index)
means = result['mean'].tolist()
mins = result['min'].tolist()
maxs = result['max'].tolist()
sums = result['sum'].tolist()

index = np.arange(n_groups)
bar_width = 0.1

rects1 = plt.bar(index, means, bar_width, color = 'r', label = 'Mean')

rects2 = plt.bar(index + bar_width, mins, bar_width, color = 'g', label = 'Min')

rects3 = plt.bar(index + bar_width * 2, maxs, bar_width, color = 'b', label = 'Max')

rects4 = plt.bar(index + bar_width * 3, sums, bar_width, color = 'y', label = 'Sum')

plt.xticks(index, result.index.tolist())
plt.legend()
plt.show()